### Notebook name: MarkRecaptureSim.ipynb
#### Author: Sreejith Menon (smenon8@uic.edu)

#### Mark Recapture Simulation Notebook

Recognize individuals that appeared on day 1 and then on day 2
Individuals that appear on day 1 are **marks**.    
If the same individuals appear on day 2 then these are **recaptures**

*By appeared it means the individuals who were photographed on day 1 as well as day 2*

To change the behavior of the script only change the values of the dictionary days. Changing days dict can filter out the images to the days the images were clicked. 

In [186]:
import json
from datetime import datetime
import DataStructsHelperAPI as DS
import importlib
importlib.reload(DS)
import GetPropertiesAPI as GP
import BuildConsolidatedFeaturesFile as BCF
importlib.reload(BCF)
import pandas as pd
import cufflinks as cf # this is necessary to link pandas to plotly
cf.go_online()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [73]:
with open("../data/imgs_exif_data_full.json","r") as inpFl:
    jsonObj = json.load(inpFl)

In [121]:
# Extract only the date information for all the given images
imgDateDict = {gid : DS.getDateFromStr(jsonObj[gid]['datetime'],'%Y-%m-%d %H:%M:%S','%Y-%m-%d') for gid in jsonObj.keys()}

# divide the days into day 1 and day 2 
days = {'2015-02-18' : '2015-02-18',
 '2015-02-19' : '2015-02-19',
 '2015-02-20' : '2015-02-20',
 '2015-02-25' : '2015-02-25',
 '2015-02-26' : '2015-02-26',
 '2015-03-01' : '2015-03-01',
 '2015-03-02' : '2015-03-02'}

# days = {'2015-03-01' : 1,
#         '2015-03-02' : 2 }

# filter out only the GIDs that were taken on either day1 on day2
filteredGid = filter(lambda x : imgDateDict[x] in days.keys(),imgDateDict.keys())

# Replace the day with Mark or Recapture
gidsDayNum = { gid : days[imgDateDict[gid]] for gid in filteredGid } 

In [122]:
with open("../data/full_gid_aid_map.json","r") as inpJson:
    gidAidMap = json.load(inpJson)

# Extract only the GIDs which are either taken on day 1 or day 2 and have a valid annotation
gidAidOfInterest = { gid : gidAidMap[gid][0] for gid in gidsDayNum.keys() if len(gidAidMap[gid][0]) != 0}

In [123]:
with open("../data/full_aid_features.json","r") as inpJson:
    aidFtr = json.load(inpJson)
    
gidNid = { gid : [aidFtr[str(aid)][0] for aid in gidAidOfInterest[gid]] for gid in gidAidOfInterest.keys() }

In [124]:
len(gidsDayNum.keys())

9285

In [125]:
len(gidNid)

6422

In [126]:
nidMarkRecap = {}
for gid in gidNid.keys():
    for nid in gidNid[gid]:
        if nid > 0:
            nidMarkRecap[nid] = nidMarkRecap.get(nid,[]) + [gidsDayNum[gid]]

In [101]:
uniqueIndsDay1 = [nid for nid in nidMarkRecap if 1 in nidMarkRecap[nid]]
uniqueIndsDay2 = [nid for nid in nidMarkRecap if 2 in nidMarkRecap[nid]]

In [193]:
len(uniqueIndsDay1),len(uniqueIndsDay2)
nidMarkRecapSet = { nid : list(set(nidMarkRecap[nid])) for nid in nidMarkRecap.keys()}

In [194]:
indsPerDay = {}
for nid in nidMarkRecapSet:
    for day in nidMarkRecapSet[nid]:
        indsPerDay[day] = indsPerDay.get(day,0) + 1

In [195]:
df1 = pd.DataFrame(indsPerDay,index=['IndsIdentified']).transpose()

fig1 = df1.iplot(kind='bar',filename='Individuals seen per day',title='Individuals seen per day')
iframe1 = fig1.embed_code

In [188]:
uniqIndsPerDay = {}
for nid in nidMarkRecapSet:
    if len(nidMarkRecapSet[nid]) == 1:
        uniqIndsPerDay[nidMarkRecapSet[nid][0]] = uniqIndsPerDay.get(nidMarkRecapSet[nid][0],0) + 1

In [189]:
df2 = pd.DataFrame(uniqIndsPerDay,index=['IndsIdentifiedOnlyOnce']).transpose()

fig2 = df2.iplot(kind='bar',filename='Individuals seen only that day',title='Individuals seen only that day')
iframe2 = fig2.embed_code

In [190]:
indsSeenFirst = {}
for nid in nidMarkRecapSet:
    indsSeenFirst[min(nidMarkRecapSet[nid])] = indsSeenFirst.get(min(nidMarkRecapSet[nid]),0) + 1

In [191]:
df3 = pd.DataFrame(indsSeenFirst,index=['FirstTimeInds']).transpose()

fig3 = df3.iplot(kind='bar',filename='Individuals first seen on that day',title='Individuals first seen on that day')
iframe3 = fig3.embed_code

In [196]:
iframe1,iframe2,iframe3

('<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~smenon8/125.embed" height="525px" width="100%"></iframe>',
 '<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~smenon8/127.embed" height="525px" width="100%"></iframe>',
 '<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~smenon8/129.embed" height="525px" width="100%"></iframe>')